In [10]:
import torch
import torch.nn.functional as F
def my_kl_loss(p, q):
    res = p * (torch.log(p + 0.0001) - torch.log(q + 0.0001))
    return torch.mean(torch.sum(res, dim=-1))

In [21]:
p = F.softmax(torch.tensor([-1.0, 2.0, 3.0]), dim=-1)
print(p)
q = F.softmax(torch.tensor([3.0, 2.0, 1]), dim=-1)
my_kl_loss(p,q)

tensor([0.0132, 0.2654, 0.7214])


tensor(1.4704)

In [22]:
torch.cuda.is_available()

True

In [23]:
import torch

# 创建一个简单的计算图
x = torch.tensor([2.0], requires_grad=True)
y = x**2
z = y + 3

# 计算梯度并进行反向传播
loss = z.sum()
loss.backward(retain_graph=True)

# 输出梯度
print("First backward:")
print("Gradient of x:", x.grad)

# 再次进行反向传播，保留计算图
loss.backward(retain_graph=True)

# 输出梯度
print("\nSecond backward:")
print("Gradient of x:", x.grad)


First backward:
Gradient of x: tensor([4.])

Second backward:
Gradient of x: tensor([8.])
